# 面向对象的 JavaScript

JavaScript 靠"原型链"（prototype chain）模式，来实现继承。

JavaScript 在一开始设计时并没有引入类（class）的概念，但是它引入了 `new` 命令，用来从原型对象生成一个实例对象。在 Javascript 中，`new` 命令后面跟的不是类，而是构造函数。

所谓"构造函数"，其实就是一个普通函数，但是内部使用了 `this` 变量。对构造函数使用 `new` 运算符，就能生成实例，并且 `this` 变量会绑定在实例对象上。如下：

In [1]:
// Dog 构造函数，表示狗对象的原型。
function Dog(name){
  this.name = name;
};

const dogA = new Dog('大毛');
console.log(dogA.name);  // 大毛
// 这时 dogA 会自动含有一个 constructor 属性，指向它的构造函数。
console.log(dogA.constructor === Dog);  // true
// instanceof 运算符，验证原型对象与实例对象之间的关系。
console.log(dogA instanceof Dog);  // true

大毛
true
true


## prototype

用构造函数生成实例对象，有一个缺点，那就是无法共享属性和方法，如下：

In [1]:
function Dog(name){
  this.name = name;
  this.species = '犬科';
};

const dogB = new Dog('大毛');
const dogC = new Dog('二毛');

dogB.species = '猫科';
console.log(dogC.species);  // 显示"犬科"，不受 dogB 的影响
console.log(dogB.species === dogC.species);  // false

犬科
false


要解决这个问题，可以为构造函数设置一个 `prototype` 属性，这个属性包含一个对象（以下简称"prototype对象"），所有实例对象需要共享的属性和方法，都放在这个对象里面；那些不需要共享的属性和方法，就放在构造函数里面。

实例对象一旦创建，将自动引用 `prototype` 对象的属性和方法。也就是说，实例对象的属性和方法，分成两种，一种是本地的，另一种是引用的。如下：

In [2]:
function Dog(name){
  this.name = name;
};

Dog.prototype = { species : '犬科' };

const dogD = new Dog('大毛');
const dogE = new Dog('二毛');

console.log(dogD.species);  // 犬科
console.log(dogE.species);  // 犬科

Dog.prototype.species = '猫科';

console.log(dogD.species);  // 猫科
console.log(dogE.species);  // 猫科
console.log(dogD.species === dogE.species)  // true
// isPrototypeOf() 方法用来判断，某个 proptotype 对象和某个实例之间的关系。
console.log(Dog.prototype.isPrototypeOf(dogD));  // true
// hasOwnProperty() 方法用来判断某一个属性到底是本地属性，还是继承自 prototype 对象的属性。
console.log(dogD.hasOwnProperty("name"));  // true
console.log(dogD.hasOwnProperty("species"));  // false
// in 运算符可以用来判断，某个实例是否含有某个属性，不管是不是本地属性。
console.log("name" in dogD);  // true
console.log("species" in dogD);  // true

犬科
犬科
猫科
猫科
true
true
true
false
true
true


由于所有的实例对象共享同一个 `prototype` 对象，那么从外界看起来，`prototype` 对象就好像是实例对象的原型，而实例对象则好像"继承"了 `prototype` 对象一样。这就是 Javascript 继承机制的设计思想。

## 继承

假设我们有如下两个构造函数，如何使 `Cat` 继承 `Animal` 呢？

In [3]:
function Animal(){
  this.species = "动物";
}

function Cat(name, color){
  this.name = name;
  this.color = color;
}

### 利用空对象作为中介以实现继承

由于 `Animal` 对象中，不变的属性都可以直接写入 `Animal.prototype`。所以，我们也可以让 `Cat()` 跳过 `Animal()`，直接继承 `Animal.prototype`。如下：

In [8]:
function Animal(){
  this.species = "动物";
}

function Cat(name,color){
  this.name = name;
  this.color = color;
}


// 首先，把 Animal 的所有不变属性，都放到它的 prototype 对象上。
function Animal(){}
Animal.prototype.species = "动物";

// 此函数利用 `F` 空对象作为中介以实现继承。
function extend(Child, Parent) {
  let F = function(){};
  F.prototype = Parent.prototype;
  Child.prototype = new F();
  Child.prototype.constructor = Child;
  // 为子对象设一个 uber 属性，这个属性直接指向父对象的 prototype 属性。
  // 这等于在子对象上打开一条通道，可以直接调用父对象的方法。
  // 这一行放在这里，只是为了实现继承的完备性，纯属备用性质。
  Child.uber = Parent.prototype;
}

extend(Cat, Animal);
let cat1 = new Cat("大毛","黄色");
console.log(cat1.species); // 动物

动物


### 拷贝继承

简单说，把父对象的所有属性和方法，拷贝进子对象，也能够实现继承。如下：

In [7]:
function Animal(){
  this.species = "动物";
}

function Cat(name,color){
  this.name = name;
  this.color = color;
}


// 首先，把 Animal 的所有不变属性，都放到它的 prototype 对象上。
function Animal(){}
Animal.prototype.species = "动物";

// 此函数将父对象的 prototype 对象中的属性，一一拷贝给 Child 对象的 prototype 对象。
function extend2(Child, Parent) {
  let p = Parent.prototype;
  let c = Child.prototype;
  for (let i in p) {
    c[i] = p[i];
  }
  c.uber = p;
}

// 使用此函数实现继承
extend2(Cat, Animal);
let cat2 = new Cat("大毛","黄色");
console.log(cat2.species); // 动物

动物


## 参考

- [Javascript继承机制的设计思想](http://www.ruanyifeng.com/blog/2011/06/designing_ideas_of_inheritance_mechanism_in_javascript.html)
- [Javascript 面向对象编程（一）：封装](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_encapsulation.html)
- [Javascript面向对象编程（二）：构造函数的继承](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_inheritance.html)
- [Javascript面向对象编程（三）：非构造函数的继承](http://www.ruanyifeng.com/blog/2010/05/object-oriented_javascript_inheritance_continued.html)
- [ECMAScript 6实现了class，对JavaScript前端开发有什么意义？](https://www.zhihu.com/question/29789315/answer/51894352)